In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

Before reading the dataset, with the following code, we say that "this dataset has latin letters and according to this knowledge, encode this dataset".

In [ ]:
whole_dataset = pd.read_csv("/kaggle/input/twitter-user-gender-classification/gender-classifier-DFE-791531.csv", encoding = "latin1")

In [ ]:
whole_dataset

I am going to use only "gender" and "text" columns, that's why I need to concat these two columns. In the following parts I will do a classification for predicting the text writer's gender.

In [ ]:
new_dataset = pd.concat([whole_dataset.gender,whole_dataset.description], axis = 1)

In [ ]:
new_dataset

In [ ]:
new_dataset.head(20)

If a row contains NaN value, delete this row:

In [ ]:
new_dataset = new_dataset.dropna(axis = 0)

In [ ]:
new_dataset.head(20)

In [ ]:
new_dataset.gender = [1 if i == "female" else 0 for i in new_dataset.gender]

In [ ]:
new_dataset.gender

In [ ]:
new_dataset.head(10)

In [ ]:
import re

In [ ]:
sentence1 = new_dataset.description[4]
print(sentence1)

In [ ]:
sentence1_after_regularexpression = re.sub("[^a-zA-Z]", " ", sentence1)

In [ ]:
sentence1_after_regularexpression

In [ ]:
sentence1_after_regularexpression = sentence1_after_regularexpression.lower()

In [ ]:
sentence1_after_regularexpression

In [ ]:
import nltk

In [ ]:
nltk.download("stopwords")

Stopwords were downloaded as corpus file and we import them after that process.

In [ ]:
from nltk.corpus import stopwords  

In [ ]:
sentence1_after_regularexpression = nltk.word_tokenize(sentence1_after_regularexpression)

In [ ]:
sentence1_after_regularexpression

In [ ]:
example_string1 = "shouldn't go away"

In [ ]:
after_splitting = example_string1.split(" ")
after_splitting

In [ ]:
after_nltk_word_tokenize = nltk.word_tokenize(example_string1)
after_nltk_word_tokenize

Using nltk library and tokenizer method split our words better.

In [ ]:
sentence1_after_regularexpression 

In [ ]:
sentence1_after_regularexpression_andstopwords = [i for i in sentence1_after_regularexpression if not i in set(stopwords.words("english"))]

In [ ]:
sentence1_after_regularexpression_andstopwords

In [ ]:
import nltk as nlp

In [ ]:
lemma = nlp.WordNetLemmatizer()
sentence1_after_regularexpression_andstopwords_andlemmatization = [lemma.lemmatize(i) for i in sentence1_after_regularexpression_andstopwords]

sentence1_after_regularexpression_andstopwords_andlemmatization

In [ ]:
last_sent = " ".join(sentence1_after_regularexpression_andstopwords_andlemmatization)

In [ ]:
last_sent

## Implementin each path for the whole dataset

In [ ]:
import re
import nltk
from nltk.corpus import stopwords  
import nltk as nlp

### Cleaning data:

In [ ]:
description_list = []

for each_sentence in new_dataset.description:
    each_sentence = re.sub("[^a-zA-Z]", " ", each_sentence)   # regular expression
    each_sentence = each_sentence.lower()                     # Lower case
    each_sentence = nltk.word_tokenize(each_sentence)         # Splitting
    each_sentence =  [i for i in each_sentence if not i in set(stopwords.words("english"))]  # Irrelevant words
    lemma = nlp.WordNetLemmatizer() 
    each_sentence = [lemma.lemmatize(i) for i in each_sentence]    # Finding root of the words
    each_sentence = " ".join(each_sentence)                        # Converting list to string
    
    description_list.append(each_sentence)
    

### Bag of Words:

![](https://miro.medium.com/max/880/1*hLvya7MXjsSc3NS2SoLMEg.png)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_features=4000)  # Most 4000 repeated words.

In [ ]:
sparce_matrix = count_vectorizer.fit_transform(description_list)
sparce_matrix = sparce_matrix.toarray()
sparce_matrix

In [ ]:
sparce_matrix.shape

Most repetitive (800) words:

In [ ]:
count_vectorizer.get_feature_names()

### Machine Learning Model:

In [ ]:
x = sparce_matrix
y = new_dataset.gender.values.reshape(-1,1)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.1, random_state = 42)

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB()
nb.fit(xtrain, ytrain)

In [ ]:
predictions = nb.predict(xtest)
print("Accuracy:", nb.score(predictions.reshape(-1,1), ytest))